# The Discrete Fourier Transform (DFT) and Fast Fourier Transform (FFT)

---

## **1. The Discrete Fourier Transform**

Given an input array $x = \{x_i, i = 0, \ldots, N-1\}$ of length $N$,  
the **Discrete Fourier Transform (DFT)** is defined as:

$$
\hat{x}_k = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_j e^{-2\pi i k j / N}, \quad k = 0, 1, \ldots, N-1.
$$

Here $i$ denotes the imaginary unit ($i^2 = -1$).

---

### **Matrix Representation**

The transformation from $x$ to $\hat{x}$ is **linear**:

$$
\hat{x} = \frac{1}{\sqrt{N}} M_N x
$$

where $M_N$ is the $N \times N$ DFT matrix:

$$
M_N =
\begin{bmatrix}
e^{-2\pi i \cdot 0 \cdot 0/N} & e^{-2\pi i \cdot 0 \cdot 1/N} & \cdots & e^{-2\pi i \cdot 0 \cdot (N-1)/N} \\
e^{-2\pi i \cdot 1 \cdot 0/N} & e^{-2\pi i \cdot 1 \cdot 1/N} & \cdots & e^{-2\pi i \cdot 1 \cdot (N-1)/N} \\
\vdots & \vdots & \ddots & \vdots \\
e^{-2\pi i \cdot (N-1) \cdot 0/N} & e^{-2\pi i \cdot (N-1) \cdot 1/N} & \cdots & e^{-2\pi i \cdot (N-1) \cdot (N-1)/N}
\end{bmatrix}
$$

---

### **Periodicity Property**

We can define $\hat{x}_k$ for any integer $k$ (not just $0,\ldots,N-1$):

$$
\hat{x}_{k+N} = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_j e^{-2\pi i (k+N)j / N}
= \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_j e^{-2\pi i k j / N} e^{-2\pi i j}
= \hat{x}_k.
$$

Hence, the DFT is **periodic** in $k$ with period $N$, i.e.,

$$
\hat{x}_k = \hat{x}_{k \bmod N}.
$$

---

## **2. The Inverse Transform**

The **inverse DFT** (IDFT) recovers $x$ from $\hat{x}$ by changing the sign in the exponent:

$$
\tilde{x}_k = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} \hat{x}_j e^{2\pi i k j / N}, \quad k = 0, 1, \ldots, N-1.
$$

Matrix form:

$$
\tilde{x} = \frac{1}{\sqrt{N}} Q_N \hat{x},
$$

where $Q_N$ is the **inverse DFT matrix**:

$$
Q_N =
\begin{bmatrix}
e^{2\pi i \cdot 0 \cdot 0/N} & e^{2\pi i \cdot 0 \cdot 1/N} & \cdots & e^{2\pi i \cdot 0 \cdot (N-1)/N} \\
e^{2\pi i \cdot 1 \cdot 0/N} & e^{2\pi i \cdot 1 \cdot 1/N} & \cdots & e^{2\pi i \cdot 1 \cdot (N-1)/N} \\
\vdots & \vdots & \ddots & \vdots \\
e^{2\pi i \cdot (N-1) \cdot 0/N} & e^{2\pi i \cdot (N-1) \cdot 1/N} & \cdots & e^{2\pi i \cdot (N-1) \cdot (N-1)/N}
\end{bmatrix}
$$

It can be shown algebraically that $\tilde{\hat{x}} = x$, confirming that $Q_N = M_N^{-1}$ up to the scaling factor.

Thus, the DFT and inverse DFT are **perfect inverses** — no information is lost.

---

### **Exercise 1**

> **Compute the DFT of small vectors:**
> 1. What is the DFT of a 2-vector in closed form?  
> 2. What about a 4-vector?

---

## **3. Periodicities — Time Domain vs. Frequency Domain**

For fixed $N$ and $k$, define a **Discrete Complex Sinusoid (DCS)** vector:

$$
\text{DCS}(k)_j = e^{2\pi i j k / N}, \quad j = 0, 1, \ldots, N-1.
$$

These are points on the unit circle; as $j$ increases, each point rotates counterclockwise by $2\pi k / N$.  
After $N$ steps, we have made $k$ complete rotations — so $k/N$ is the **frequency** of the sinusoid.

---

### **Column Representation**

The columns of $Q_N$ are precisely these discrete complex sinusoids:

$$
Q_N = [\text{DCS}(0), \text{DCS}(1), \ldots, \text{DCS}(N-1)].
$$

Then:

$$
x = Q_N \hat{x} = \sum_{j=0}^{N-1} \hat{x}_j \, \text{DCS}(j).
$$

Interpretation:
- $x_j$ are values in the **time domain**.
- $\hat{x}_j$ are values in the **frequency domain**.

Each $\hat{x}_j$ tells how much of frequency $j/N$ is present in $x$.

---

## **4. Computing the DFT Efficiently — The Fast Fourier Transform (FFT)**

Directly computing the DFT from its definition requires $\mathcal{O}(N^2)$ operations (since each of $N$ outputs sums $N$ terms).

The **Cooley–Tukey FFT algorithm** reduces this to $\mathcal{O}(N \log N)$ by exploiting symmetry when $N$ is a power of 2.

We now outline the recursive structure.

---

### **Even–Odd Decomposition**

Let:

$$
x_E = [x_0, x_2, x_4, \ldots, x_{N-2}], \quad
x_O = [x_1, x_3, x_5, \ldots, x_{N-1}].
$$

Split the DFT sum:

$$
\begin{aligned}
\hat{x}_k
&= \sum_{j=0}^{N/2-1} x_{2j} e^{-2\pi i k (2j)/N} + \sum_{j=0}^{N/2-1} x_{2j+1} e^{-2\pi i k (2j+1)/N} \\
&= \sum_{j=0}^{N/2-1} x_E[j] e^{-2\pi i k j / (N/2)} + e^{-2\pi i k / N} \sum_{j=0}^{N/2-1} x_O[j] e^{-2\pi i k j / (N/2)} \\
&= \hat{x}_E[k] + e^{-2\pi i k / N} \hat{x}_O[k].
\end{aligned}
$$

This formula holds for $k = 0, \ldots, N/2 - 1$, and due to periodicity:

$$
\hat{x}_{k+N/2} = \hat{x}_E[k] - e^{-2\pi i k / N} \hat{x}_O[k].
$$

---

### **Matrix Factorization**

Let:
- $A_{N/2}$ = diagonal matrix with diagonal entries $e^{-2\pi i k / N}$ for $k = 0, \ldots, N/2 - 1$.
- $B_N$ = permutation matrix that reorders $x$ as $[x_0, x_2, \ldots, x_{N-2}, x_1, x_3, \ldots, x_{N-1}]$.
- $I_N$ = identity matrix, $O_N$ = zero matrix.

Then:

$$
\hat{x} =
\begin{bmatrix}
I_{N/2} & A_{N/2} \\
I_{N/2} & -A_{N/2}
\end{bmatrix}
\begin{bmatrix}
M_{N/2} & O_{N/2} \\
O_{N/2} & M_{N/2}
\end{bmatrix}
B_N x
$$

This factorization shows that computing $M_N x$ (the DFT) can be broken into smaller DFTs of size $N/2$, then recombined.

---

By repeating this factorization recursively, we reach the base case:

$$
M_2 =
\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}.
$$

At each recursion level, we halve the size of the problem.

---

### **Complexity**

At each step, we perform a constant number of multiplications and additions per data point, and there are $\log_2 N$ levels.

Hence, the FFT reduces computation from $\mathcal{O}(N^2)$ to $\mathcal{O}(N \log N)$.

---

### **Sparse Matrix Interpretation**

Each intermediate matrix in the FFT factorization has **only two nonzero entries per row**, so multiplication costs are linear in $N$.  
This sparsity is what enables the dramatic efficiency improvement.

---

**Exercise 1**

What normalization does numpy's fft use?

This can be answered by computing the FFT of the vector x=(1,0,...,0). If we multiply by the matrix whose j,k entry is $e^{-2\pi ijk/N}$ we should get
the vector of all 1's.

In [4]:
import numpy as np
for i in range(1,10):
    x=np.zeros((i))
    x[0]=1
    print(np.fft.fft(x))

[1.+0.j]
[1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]


We see that the normalization constant is taken to be 1.

What about the inverse transform? If we multiply by the matrix whose j,k entry is $e^{2\pi ijk/N}$ we should get
the vector of all 1's.

In [5]:
import numpy as np
for i in range(1,10):
    x=np.zeros((i))
    x[0]=1
    print(np.fft.ifft(x))

[1.+0.j]
[0.5+0.j 0.5+0.j]
[0.33333333+0.j 0.33333333+0.j 0.33333333+0.j]
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
[0.2+0.j 0.2+0.j 0.2+0.j 0.2+0.j 0.2+0.j]
[0.16666667+0.j 0.16666667+0.j 0.16666667+0.j 0.16666667+0.j
 0.16666667+0.j 0.16666667+0.j]
[0.14285714+0.j 0.14285714+0.j 0.14285714+0.j 0.14285714+0.j
 0.14285714+0.j 0.14285714+0.j 0.14285714+0.j]
[0.125+0.j 0.125+0.j 0.125+0.j 0.125+0.j 0.125+0.j 0.125+0.j 0.125+0.j
 0.125+0.j]
[0.11111111+0.j 0.11111111+0.j 0.11111111+0.j 0.11111111+0.j
 0.11111111+0.j 0.11111111+0.j 0.11111111+0.j 0.11111111+0.j
 0.11111111+0.j]


Here we see that after multiplying by the matrix numpy divides by $N$ the length of the input array. This should not be terribly surprising because we know we needed to divide by $\sqrt{N}$ in our definition of the DFT and again in the definition of the inverse. 

**Exercise 2**

Using Euler's formula
$$
M_2 = \left[
\begin{array}{cc}
e^{-2\pi i 0\cdot 0/2} & e^{-2\pi i 0\cdot 1/2} \\
e^{-2\pi 1 i \cdot 0/2} & e^{-2\pi i 1\cdot 1/2} \\ 
\end{array}\right] =
\left[
\begin{array}{cc}
e^{0} & e^{0} \\
e^{0} & e^{-\pi i} \\ 
\end{array}\right]
=
\left[
\begin{array}{cc}
1 & 1 \\
1 & -1 \\ 
\end{array}\right]
$$

so the DFT of $[x_0,x_1]$ is 
$$
[\frac{x_0+x+1}{\sqrt{2}},\frac{x_0-x_1}{\sqrt{2}}]
$$

$$
M_4 = \left[
\begin{array}{cccc}
e^{-2\pi i 0\cdot 0/4} & e^{-2\pi i 0\cdot 1/4} &e^{-2\pi i 0\cdot 2/4} & e^{-2\pi i 0\cdot 3/4} \\
e^{-2\pi i 1\cdot 0/4} & e^{-2\pi i 1\cdot 1/4} & e^{-2\pi i 1\cdot 2/4} & e^{-2\pi i  1\cdot 3/4}\\ 
e^{-2\pi i 2\cdot 0/4} & e^{-2\pi i 2\cdot 1/4} &e^{-2\pi i 2\cdot 2/4} & e^{-2\pi i 2\cdot 3/4} \\
e^{-2\pi i 3\cdot 0/4} & e^{-2\pi i 3\cdot 1/4} & e^{-2\pi i  3\cdot 2/4} & e^{-2\pi i 3\cdot 3/4}\\ 
\end{array}\right] =
\left[
\begin{array}{cccc}
e^{0} & e^{0} & e^{0} & e^{0}\\
e^{0} & e^{-\pi i/2} & e^{-\pi i} & e^{-3\pi i/2}\\ 
e^{0} & e^{-\pi i} & e^{-2 \pi i} & e^{-3\pi i}\\ 
e^{0} & e^{-3\pi i/2} & e^{-3 pi i} & e^{-9\pi i/2}\\ 
\end{array}\right]
=
\left[
\begin{array}{cccc}
1 & 1 & 1 & 1\\
1 & -i & -1 & i \\ 
1 & -1 & 1 & -1 \\ 
1 & i & -1 & -i \\ 
\end{array}\right]
$$

so the DFT of $[x_0,x_1,x_2,x_3]$ is 
$$
[\frac{x_0+x_1+x_2+x_3}{2},\frac{x_0-i x_1-x_2 + i x_3}{2},
\frac{x_0-x_1+x_2-x_3}{2},\frac{x_0+i x_1-x_2 - i x_3}{2}]
$$


**Exercise 3**

Create a function taking inputs 

- N = length of x
- k = nonegative integer
- x = input numpy 1-d array of length N,

and outputs 

- the product of ${\cal }B^{(k)}_N$ and a given $N$-vector $x$ where $N$ is a power of 2. Here ${\cal }B^{(k)}_N$ is $N \times N$ having $2^{k-1}$ diagonal blocks each being $B_{N/2^{k-1}}$ which is $N/2^{k-1} \times N/2^{k-1}.$ Each such diagonal block has the effect of taking $x_0,\ldots,x_{m-1}$ and replacing it with $x_0,x_2,...x_{m-2},x_1,x_3,...,x_{m-1}$ where $m=N/2^{k-1}.$

In [6]:
import numpy as np
def MultiplyByB(N,k,x):
    xnew=[]
    nblocks=2**(k-1)
    #print("nblocks = "+str(nblocks))
    blocksize=int(N/2**(k-1))
    #print("blocksize = "+str(blocksize))
    
    chunksize=int(blocksize/2)
    for i in range(nblocks):
        shift=i*blocksize
        for j in range(chunksize):
            xnew.append(x[shift+2*j])
        for j in range(chunksize):
            xnew.append(x[shift+2*j+1])
    return(xnew)
x=list(range(16))
x=MultiplyByB(16,1,x)
print(x)
x=MultiplyByB(16,2,x)
print(x)
x=MultiplyByB(16,3,x)
print(x)

[0, 2, 4, 6, 8, 10, 12, 14, 1, 3, 5, 7, 9, 11, 13, 15]
[0, 4, 8, 12, 2, 6, 10, 14, 1, 5, 9, 13, 3, 7, 11, 15]
[0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]


**Exercise 4**

Create a function that computes and returns the product of the $N \times N$ block diagonal matrix 
consisting of $N/2$ diagonal blocks, each of $2 \times 2$ matrix of the form 
$\left[\begin{array}{cc}
 1 & 1 \\
 1 & -1 \\
 \end{array}\right].$

In [7]:
def MultiplyByM(N,x):
    xnew=[]
    for i in range(int(N/2)):
        xnew.append(x[2*i]+x[2*i+1])
        xnew.append(x[2*i]-x[2*i+1])
    return(xnew)
x=list(range(8))
MultiplyByM(8,x)

[1, -1, 5, -1, 9, -1, 13, -1]

**Exercuse 5**

Write a function that computes and returns the product of ${\cal }A^{(k)}_N$ and a given $N$ vector $x$ where $N$ is a power of 2. Here ${\cal }A^{(k)}_N$ is $N \times N$ having $2^{k-1}$ diagonal blocks each being of the form
$$\left[\begin{array}{cc}
 I_{N/2^k} & A_{N/2^k} \\
 I_{N/2^k} & -A_{N/2^k}\\
 \end{array}\right].
$$
and where $A_{p}$ denotes a diagonal $p\times p$ matrix with diagonal entries $e^{2\pi ij/p}, p=0,1,\ldots,p-1.$

In [9]:
def MultiplyByA(N,k,x):
    xnew=[]
    blocksize=int(N/2**(k-1))
    nblocks=2**(k-1)
    halfblocksize=int(blocksize/2)
    for i in range(nblocks):
        shift=blocksize*i
        for j in range(halfblocksize):
            xp=x[shift+j]+np.exp(-2*np.pi*1j*j/blocksize)*x[shift+halfblocksize+j]
            xnew.append(xp)
        for j in range(halfblocksize):
            #print(shift+j)
            #print(shift+halfblocksize+j)
            xp=x[shift+j]-np.exp(-2*np.pi*1j*j/blocksize)*x[shift+halfblocksize+j]
            xnew.append(xp)
    return(xnew)

**Exercise 6**

Use the functions defined above to create an FFT function that works when $x$ is a vector whose length is a power of 2.

In [10]:
import numpy as np
def FFT(x):
    N=len(x)
    p=int(np.log2(N))
    for k in range(1,p):
        x=MultiplyByB(N,k,x)
    x=MultiplyByM(N,x)
    for k in range(p-1,0,-1):
        x=MultiplyByA(N,k,x)
    return(x)

Compare with numpy's fft.

In [11]:
x=np.random.normal(0,1,16)
xhat1=FFT(x)
xhat2=np.fft.fft(x)
xhat1/xhat2

array([1.-0.00000000e+00j, 1.+1.71121874e-17j, 1.+8.99905803e-17j,
       1.-1.45536565e-17j, 1.+6.02748275e-17j, 1.+3.13925075e-17j,
       1.+9.51509140e-17j, 1.+1.18018803e-16j, 1.+0.00000000e+00j,
       1.+1.92646851e-16j, 1.+5.69073761e-17j, 1.+8.37010789e-17j,
       1.+6.34288608e-17j, 1.+7.29613195e-17j, 1.+6.26186002e-18j,
       1.-1.71121874e-17j])

**Exercise 7**

Function to create the $N\times N$ DFT matrix.

In [12]:
def DFTMatrix(N):
    M=np.zeros((N,N),dtype=complex)
    for i in range(N):
        for j in range(N):
            M[i,j]=np.exp(-2*np.pi*(0.+1j)*i*j/N)
    return(M)
M=DFTMatrix(16)
xhat3=np.dot(M,x)
xhat3/xhat2

array([1.-0.00000000e+00j, 1.+2.96294193e-16j, 1.+4.59067011e-16j,
       1.-3.56608530e-16j, 1.+6.30239513e-16j, 1.+8.91238636e-16j,
       1.+9.13999678e-16j, 1.+5.87147581e-16j, 1.-2.34695186e-15j,
       1.+2.70350392e-15j, 1.+1.26243157e-15j, 1.+6.47162535e-16j,
       1.+1.53859592e-15j, 1.-8.08217948e-15j, 1.+4.09650123e-15j,
       1.+4.52647231e-15j])

**Exercise 8** 

Compare timings.

In [62]:
import timeit

setup1="""
import numpy as np
np.random.seed(18912)
p=12
N=2**p
x=np.random.normal(0,1,N)
"""

code1="""
xhat=np.fft.fft(x)
"""

res=timeit.timeit(stmt=code1,setup=setup1,number=10)/10
print(res)


setup2="""
import numpy as np
np.random.seed(18912)
p=12
N=2**p
x=np.random.normal(0,1,N)
def MultiplyByB(N,k,x):
    xnew=[]
    nblocks=2**(k-1)
    #print("nblocks = "+str(nblocks))
    blocksize=int(N/2**(k-1))
    #print("blocksize = "+str(blocksize))
    
    chunksize=int(blocksize/2)
    for i in range(nblocks):
        shift=i*blocksize
        for j in range(chunksize):
            xnew.append(x[shift+2*j])
        for j in range(chunksize):
            xnew.append(x[shift+2*j+1])
    return(xnew)
def MultiplyByM(N,x):
    xnew=[]
    for i in range(int(N/2)):
        xnew.append(x[2*i]+x[2*i+1])
        xnew.append(x[2*i]-x[2*i+1])
    return(xnew)
def MultiplyByA(N,k,x):
    xnew=[]
    blocksize=int(N/2**(k-1))
    nblocks=2**(k-1)
    halfblocksize=int(blocksize/2)
    for i in range(nblocks):
        shift=blocksize*i
        for j in range(halfblocksize):
            xp=x[shift+j]+np.exp(-2*np.pi*1j*j/blocksize)*x[shift+halfblocksize+j]
            xnew.append(xp)
        for j in range(halfblocksize):
            #print(shift+j)
            #print(shift+halfblocksize+j)
            xp=x[shift+j]-np.exp(-2*np.pi*1j*j/blocksize)*x[shift+halfblocksize+j]
            xnew.append(xp)
    return(xnew)
def FFT(x):
    N=len(x)
    p=int(np.log2(N))
    for k in range(1,p):
        x=MultiplyByB(N,k,x)
    x=MultiplyByM(N,x)
    for k in range(p-1,0,-1):
        x=MultiplyByA(N,k,x)
    return(x)
"""

code2="""
xhat=FFT(x)
"""

res=timeit.timeit(stmt=code2,setup=setup2,number=10)/10
print(res)

setup3="""
import numpy as np
np.random.seed(18912)
p=12
N=2**p
x=np.random.normal(0,1,N)
def DFTMatrix(N):
    M=np.zeros((N,N),dtype=complex)
    for i in range(N):
        for j in range(N):
            M[i,j]=np.exp(-2*np.pi*(0.+1j)*i*j/N)
    return(M)
"""

code3="""
M=DFTMatrix(N)
xhat=np.dot(M,x)
"""

res=timeit.timeit(stmt=code3,setup=setup3,number=10)/10
print(res)






5.097000394016504e-05
0.08918740001972765
0.008870700001716613


Try to make my functions faster.

In [63]:
def MultiplyByM(N, x):
    x = np.asarray(x, dtype=complex)
    x_even = x[::2]
    x_odd = x[1::2]
    return np.concatenate([x_even + x_odd, x_even - x_odd])
def MultiplyByA(N, k, x):
    x = np.asarray(x, dtype=complex)
    blocksize = N // (2**(k - 1))
    half = blocksize // 2
    result = np.empty(N, dtype=complex)
    for i in range(0, N, blocksize):
        x0 = x[i:i+half]
        x1 = x[i+half:i+blocksize]
        w = np.exp(-2j * np.pi * np.arange(half) / blocksize)
        result[i:i+half] = x0 + w * x1
        result[i+half:i+blocksize] = x0 - w * x1
    return result
def MultiplyByB(N, k, x):
    x = np.asarray(x, dtype=complex)
    blocksize = N // (2**(k - 1))
    chunksize = blocksize // 2
    result = np.empty(N, dtype=complex)
    for i in range(0, N, blocksize):
        even_indices = np.arange(chunksize) * 2 + i
        odd_indices = even_indices + 1
        result[i:i+chunksize] = x[even_indices]
        result[i+chunksize:i+blocksize] = x[odd_indices]
    return result
def FFT(x):
    x = np.asarray(x, dtype=complex)
    N = len(x)
    p = int(np.log2(N))
    for k in range(1, p):
        x = MultiplyByB(N, k, x)
    x = MultiplyByM(N, x)
    for k in range(p - 1, 0, -1):
        x = MultiplyByA(N, k, x)
    return x

In [79]:
import timeit

setup1="""
import numpy as np
np.random.seed(18912)
p=10
N=2**p
x=np.random.normal(0,1,N)
"""

code1="""
xhat=np.fft.fft(x)
"""

res=timeit.timeit(stmt=code1,setup=setup1,number=10)/10
print(res)


setup2="""
import numpy as np
np.random.seed(18912)
p=10
N=2**p
x=np.random.normal(0,1,N)
def MultiplyByM(N, x):
    x = np.asarray(x, dtype=complex)
    x_even = x[::2]
    x_odd = x[1::2]
    return np.concatenate([x_even + x_odd, x_even - x_odd])
def MultiplyByA(N, k, x):
    x = np.asarray(x, dtype=complex)
    blocksize = N // (2**(k - 1))
    half = blocksize // 2
    result = np.empty(N, dtype=complex)
    for i in range(0, N, blocksize):
        x0 = x[i:i+half]
        x1 = x[i+half:i+blocksize]
        w = np.exp(-2j * np.pi * np.arange(half) / blocksize)
        result[i:i+half] = x0 + w * x1
        result[i+half:i+blocksize] = x0 - w * x1
    return result
def MultiplyByB(N, k, x):
    x = np.asarray(x, dtype=complex)
    blocksize = N // (2**(k - 1))
    chunksize = blocksize // 2
    result = np.empty(N, dtype=complex)
    for i in range(0, N, blocksize):
        even_indices = np.arange(chunksize) * 2 + i
        odd_indices = even_indices + 1
        result[i:i+chunksize] = x[even_indices]
        result[i+chunksize:i+blocksize] = x[odd_indices]
    return result
def FFT(x):
    x = np.asarray(x, dtype=complex)
    N = len(x)
    p = int(np.log2(N))
    for k in range(1, p):
        x = MultiplyByB(N, k, x)
    x = MultiplyByM(N, x)
    for k in range(p - 1, 0, -1):
        x = MultiplyByA(N, k, x)
    return x
"""

code2="""
xhat=FFT(x)
"""

res=timeit.timeit(stmt=code2,setup=setup2,number=10)/10
print(res)

setup3="""
import numpy as np
np.random.seed(18912)
p=10
N=2**p
x=np.random.normal(0,1,N)
def DFTMatrix(N):
    M=np.zeros((N,N),dtype=complex)
    for i in range(N):
        for j in range(N):
            M[i,j]=np.exp(-2*np.pi*(0.+1j)*i*j/N)
    return(M)
"""

code3="""
M=DFTMatrix(N)
xhat=np.dot(M,x)
"""

res=timeit.timeit(stmt=code3,setup=setup3,number=10)/10
print(res)






1.246000174432993e-05
0.004234919999726116
2.265611799992621


In [82]:
import timeit

setup1="""
import numpy as np
np.random.seed(18912)
p=10
N=2**p
x=np.random.normal(0,1,N)
"""

code1="""
xhat=np.fft.fft(x)
"""

res=timeit.timeit(stmt=code1,setup=setup1,number=10)/10
print(res)


setup2="""
import numpy as np
np.random.seed(18912)
p=10
N=2**p
x=np.random.normal(0,1,N)
def bit_reversed_indices(n):
    p = int(np.log2(n))
    indices = np.arange(n)
    reversed_indices = np.zeros(n, dtype=int)
    for i in range(n):
        b = format(i, f'0{p}b')  # binary representation
        reversed_indices[i] = int(b[::-1], 2)
    return reversed_indices

def FFT_single_function(x):
    x = np.asarray(x, dtype=complex)
    N = x.shape[0]
    assert (N & (N - 1)) == 0, "Size must be a power of 2"

    # Bit-reversal permutation
    x = x[bit_reversed_indices(N)]

    # Cooley-Tukey FFT
    m = 2
    while m <= N:
        half = m // 2
        w_m = np.exp(-2j * np.pi * np.arange(half) / m)
        for k in range(0, N, m):
            t = w_m * x[k+half:k+m]
            u = x[k:k+half]
            x[k:k+half] = u + t
            x[k+half:k+m] = u - t
        m *= 2

    return x


"""

code2="""
xhat=FFT_single_function(x)
"""

res=timeit.timeit(stmt=code2,setup=setup2,number=10)/10
print(res)






3.374000079929829e-05
0.0021311100106686355
